In [ ]:
"""
Created on Fri Jul 09 09:39 2021

This script is to transform the mean profiles to 2D fields to accelerate the tuning process

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf
import multimelt.melt_functions as meltf
from scipy import stats
from dask import delayed

import distributed
import glob

from tqdm.notebook import tqdm


In [ ]:
%matplotlib qt5

In [ ]:
client = distributed.Client(n_workers=18,dashboard_address=':8795', local_directory='/tmp', memory_limit='4GB')

In [ ]:
client

READ IN DATA

In [ ]:
mod = 'CNRM-CM6-1' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'
to2300 = False

if scenario == 'historical':
    yystart = 1850
    yyend = 2014
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100        

In [ ]:
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
inputpath_mask='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

chunk_size = 500

Files for param

In [ ]:
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_isf+'BedMachine_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf).chunk(chunks={'x': chunk_size, 'y': chunk_size})



In [ ]:
BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v3_aggregated4km_allvars.nc')
BedMachine_orig_cut = BedMachine_orig.sel(x=file_isf.x, y=file_isf.y)

ice_draft_pos = (BedMachine_orig_cut['thickness'] - BedMachine_orig_cut['surface']).where(file_isf['ISF_mask'] > 1)
ice_draft_isf = ice_draft_pos.where(file_isf['ISF_mask'] == file_isf.Nisf)

In [ ]:
tt

In [ ]:
for tt in tqdm(range(yystart, yyend+1)): #continue at 1989

    file_T_orig = xr.open_mfdataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc', chunks={'Nisf': 1})
    file_S_orig = xr.open_mfdataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc', chunks={'Nisf': 1})
    file_TS_orig = xr.merge([file_T_orig,file_S_orig])
    file_TS_orig = file_TS_orig.assign_coords({'z': -1*file_TS_orig['z'].values}).rename({'z':'depth'})
    file_TS = file_TS_orig.sel(Nisf=rignot_isf).chunk(chunks={'depth': 20})
    
    filled_TS = file_TS.ffill(dim='depth')
    
    depth_of_int = ice_draft_isf.where(ice_draft_isf<file_isf['front_bot_depth_max'], 
                                       file_isf['front_bot_depth_max']).chunk(chunks={'Nisf':1})  
    T_isf = filled_TS['thetao'].interp({'depth': depth_of_int}).drop('depth')
    
    #print('here1')
    T_isf = T_isf.where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    T_isf = T_isf.to_dataset(name='theta_in')
    #print('here2')
    T_isf['salinity_in'] = filled_TS['so'].interp({'depth': depth_of_int}).drop('depth')
    T_isf['salinity_in'] = T_isf['salinity_in'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')#.where(depth_of_int>0)
    #print('here3')
    depth_of_int = depth_of_int.where(file_isf['ISF_mask'] == file_isf.Nisf)
    depth_of_int = depth_of_int.where(file_isf['ISF_mask'] > 1).sum('Nisf')
    depth_of_int = depth_of_int.where(depth_of_int>0)
    T_isf['freezing_T'] = meltf.freezing_temperature(T_isf['salinity_in'], -1*depth_of_int)
    #print('here4')
    T_isf['thermal_forcing'] = T_isf['theta_in'] - T_isf['freezing_T']
    T_isf['depth_of_int'] = depth_of_int
    T_isf = T_isf.where(T_isf['salinity_in']>0)
    
    
    T_isf.to_netcdf(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc','w')
    del T_isf
    del depth_of_int

In [ ]:
tt = 1950
ds_tt = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc')

In [ ]:
ds_tt['theta_in'].plot()

In [ ]:
TS_list = []
for tt in range(yy_start,yy_end+1):
    ds_tt = xr.open_dataset(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    TS_list.append(ds_tt)
TS_ds = xr.concat(TS_list, dim='time')
TS_ds.to_netcdf(inputpath_profiles+'T_S_2D_fields_isf_draft_'+mod+'_'+scenario+'_allyy.nc')